In [1]:
# Computes tract-specific metrics (voxel-wise, distance-wise, tract-wise) based on
# population diffusion & tractopgraphy statistics

# Requires that BedpostX and ProbtrackX for the network of interest have already
# been computed


%load_ext autoreload
%autoreload 2

In [13]:
# Read config file
# Set up variables

import json
import csv
import os
import shutil
import glob

import utils 

import math
import nibabel as nib
import numpy as np
from scipy import stats
from nilearn import image, masking

from tqdm import tnrange, tqdm_notebook

config_file = 'config_tracts_wwn.json'

def _json_object_hook(d): return namedtuple('X', d.keys())(*d.values())
def json2obj(data): return json.loads(data, object_hook=_json_object_hook)

with open(config_file, 'r') as myfile:
    json_string=myfile.read()

params = json.loads(json_string)

params_gen = params['general']
source_dir = params_gen['source_dir']
project_dir = os.path.join(source_dir, params_gen['project_dir'])
tracts_dir = os.path.join(project_dir, params_gen['tracts_dir'], params_gen['network_name'])
rois_dir = os.path.join(source_dir, params_gen['rois_dir'], params_gen['network_name'])

if params_gen['debug']:
    debug_dir = os.path.join(tracts_dir, 'debug')
    if not os.path.isdir(debug_dir):
        os.makedirs(debug_dir)

avr_dir = os.path.join(tracts_dir, 'average')
if not os.path.isdir(avr_dir):
    os.makedirs(avr_dir)
#     print('  *Error: Averages have not yet been computed! Stopping.')
#     raise
    
dist_dir = os.path.join(tracts_dir, 'dist')
if not os.path.isdir(dist_dir):
    os.makedirs(dist_dir)
polyline_dir = os.path.join(tracts_dir, 'polylines')
if not os.path.isdir(polyline_dir):
    os.makedirs(polyline_dir)
gauss_dir = os.path.join(tracts_dir, 'gaussians')
if not os.path.isdir(gauss_dir):
    os.makedirs(gauss_dir)
gauss_fail_dir = os.path.join(tracts_dir, 'gaussians.failed')
if not os.path.isdir(gauss_fail_dir):
    os.makedirs(gauss_fail_dir)
final_dir = os.path.join(tracts_dir, 'final')
if not os.path.isdir(final_dir):
    os.makedirs(final_dir)
final_fail_dir = os.path.join(tracts_dir, 'final.failed')
if not os.path.isdir(final_fail_dir):
    os.makedirs(final_fail_dir)
log_dir = os.path.join(tracts_dir, 'log')
if not os.path.isdir(log_dir):
    os.makedirs(log_dir)


if not os.path.isdir(tracts_dir):
    os.makedirs(tracts_dir)

# Subjects
subjects = [];
with open(params_gen['subjects_file']) as subj_file:
    reader = csv.reader(subj_file)
    for row in reader:
        subjects.append(row[0])
        
subjects = sorted(subjects)
print(subjects)

# ROIs
# Read ROI list
rois = []
with open(params_gen['rois_list'],'r') as roi_file:
    reader = csv.reader(roi_file)
    for row in reader:
        rois.append(row[0])

# Define volume for save operations
        
# Determine proper suffix
roi_suffix = 'nii'
roi = rois[0]
roi_file = '{0}/{1}.{2}'.format(rois_dir, roi, roi_suffix)
if not os.path.isfile(roi_file):
    roi_suffix = 'nii.gz'
    roi_file = '{0}/{1}.{2}'.format(rois_dir, roi, roi_suffix)
    if not os.path.isfile(roi_file):
        print('No ROI file found: {0}.'.format(roi_file))
        
V_img = nib.load(roi_file)
V_img.header.set_data_dtype(np.float32)

['A00008326', 'A00010893', 'A00013809', 'A00021039', 'A00023510', 'A00025566', 'A00027651', 'A00028150', 'A00028184', 'A00028185', 'A00028287', 'A00028340', 'A00028352', 'A00028389', 'A00028399', 'A00028429', 'A00028678', 'A00028753', 'A00028784', 'A00028842', 'A00028844', 'A00028845', 'A00028912', 'A00028994', 'A00029076', 'A00029092', 'A00029104', 'A00029215', 'A00029230', 'A00029231', 'A00030912', 'A00030913', 'A00030947', 'A00030989', 'A00031167', 'A00031216', 'A00031217', 'A00031459', 'A00031794', 'A00031871', 'A00032010', 'A00033011', 'A00033021', 'A00033232', 'A00033247', 'A00033248', 'A00033589', 'A00033609', 'A00033640', 'A00033647', 'A00033673', 'A00033735', 'A00033747', 'A00033748', 'A00033752', 'A00033764', 'A00033834', 'A00033849', 'A00034193', 'A00034827', 'A00034987', 'A00035036', 'A00035363', 'A00035437', 'A00035440', 'A00035504', 'A00035535', 'A00035562', 'A00035608', 'A00035625', 'A00035827', 'A00035840', 'A00035881', 'A00035960', 'A00037110', 'A00037111', 'A00037112'

In [4]:
# Bidirectional average tract counts (normalized & binarized) from ProbtrackX output

# Description:
# For all subdirectories A in source_dir, obtains
# the minimal tract count from ROI A->B and B->A. Next normalizes these tract 
# counts and finds the average of these across subjects. Also binarizes at the 
# proportion bin_thresh (0 to 1), and finds the average binarized value
# across subjects. Write the results to target_dir.

print('\n== Computing average tracts ==\n')

params_avt = params['average_tracts']
fwhm = params_avt['fwhm']
bin_thres = params_avt['bin_thres']

avr_dir = os.path.join(tracts_dir, 'average')
if not os.path.isdir(avr_dir):
    os.makedirs(avr_dir)

V = np.zeros(V_img.shape)
V_min_avr = {}
V_bin_avr = {}

row_i = {}
roi_pairs = {}

if 'roi_pairs' in params_gen and len(params_gen['roi_pairs']) > 0:
    with open(params_gen['roi_pairs'], 'r', newline='') as pairs_file:
        for line in pairs_file:
            parts = line.strip().split(',')
            if parts[0] in V_min_avr:
                row_i = V_min_avr[parts[0]]
                roi_pairs[parts[0]].append(parts[1])
            else:
                row_i = {}
                roi_pairs[parts[0]] = [parts[1]]
            row_i[parts[1]] = np.squeeze(V)
            V_min_avr[parts[0]] = row_i
            V_bin_avr[parts[0]] = row_i.copy()
else:    

    # Instantiate list of roi pair volumes
    for i in range (0,len(rois)):
        roi_i = rois[i]
        row_i = {}
        roi_pairs[roi_i] = []
        for j in range (i+1,len(rois)):
            roi_j = rois[j]
            roi_pairs[roi_i].append(roi_j)
            row_i[roi_j] = np.squeeze(V)
        V_min_avr[roi_i] = row_i
        V_bin_avr[roi_i] = row_i.copy()

print('Processing subject-wise tractography...')
subj_passed = []  

print(subjects)

s = 1
for subject in tqdm_notebook(subjects, 'Progress'):
    prefix_sub = '{0}{1}'.format(params_gen['prefix'], subject)
    print('  Processing subject: {0} ({1} of {2})'.format(subject, s, len(subjects)))
    subject_dir = os.path.join(project_dir, params_gen['deriv_dir'], prefix_sub, \
                               params_gen['sub_dirs'], params_gen['dwi_dir'], \
                               'probtrackX', params_gen['network_name'])
#     subject_dir = os.path.join(project_dir, params_gen['deriv_dir'], params_gen['dwi_dir'],  \
#                                prefix_sub, params_gen['sub_dirs'], \
#                                'probtrackX', params_gen['network_name'])
    passed = False
#     for i in range (0,len(rois)):
#         roi_i = rois[i]
#         for j in range (i+1,len(rois)):
        
#             roi_j = rois[j]
    for roi_i in roi_pairs:
        for roi_j in roi_pairs[roi_i]:
            try:
                V_ij = nib.load('{0}/{1}/target_paths_{2}.nii.gz'.format(subject_dir, roi_i, roi_j)).get_data()
                V_ji = nib.load('{0}/{1}/target_paths_{2}.nii.gz'.format(subject_dir, roi_j, roi_i)).get_data()
                V_min = np.minimum(V_ij,V_ji) # Minimum of A->B and B->A
                mx = V_min.max()
                if mx > 0:
                    V_min = np.divide(V_min, mx) # Normalize to maximum (if not all zeros)      
                V_min = np.multiply(V_min, np.greater(V_min, bin_thres).astype(np.float16)) # Binarize to bin_thresh
                V_bin = np.greater(V_min, 0).astype(np.float16)                           
                V_min_avr[roi_i][roi_j] = np.add(V_min_avr[roi_i][roi_j], V_min)
                V_bin_avr[roi_i][roi_j] = np.add(V_bin_avr[roi_i][roi_j], V_bin)
                passed = True
            except Exception as e:
                print('  ** Error processing subject {0} [skipping]: {1}'.format(subject, e))
                break

    if passed:
        subj_passed.append(subject)
        s = s + 1
                                
# Write passed subjects to file
subjects = subj_passed
with open('{0}/subjects_avr_{1}.list'.format(tracts_dir, params_gen['network_name']), 'w') as fileout:
    for subject in subjects:
        fileout.write('{0}\n'.format(subject))
            
# Divide sums
for roi_i in roi_pairs:
    for roi_j in roi_pairs[roi_i]:
        roi_j = rois[j]
        V_min_avr[roi_i][roi_j] = np.divide(V_min_avr[roi_i][roi_j], len(subjects))
        V_bin_avr[roi_i][roi_j] = np.divide(V_bin_avr[roi_i][roi_j], len(subjects))

print('Done processing subjects.\nSmoothing averages and saving...')
        
# Normalize, smooth, and save results
for roi_i in roi_pairs:
    for roi_j in roi_pairs[roi_i]:
        
        print('  Smoothing tract for {0} | {1}'.format(roi_i, roi_j))
        
        V = V_min_avr[roi_i][roi_j]
        # Normalize
        mx = np.max(V)
        if (mx > 0):
            V = np.divide(V, mx)
        img = nib.Nifti1Image(V, V_img.affine, V_img.header)
        # Smooth
        V_img = image.smooth_img(img, fwhm)
        # Save
        nib.save(V_img, '{0}/avr_min_tract_counts_{1}_{2}.nii.gz'.format(avr_dir, roi_i, roi_j))
        
        V = V_bin_avr[roi_i][roi_j]
        # Normalize
        mx = np.max(V)
        if (mx > 0):
            V = np.divide(V, mx)
        V_img = nib.Nifti1Image(V, V_img.affine, V_img.header)
        # Smooth
        V_img = image.smooth_img(img, fwhm)
        # Save
        nib.save(img, '{0}/avr_bin_tract_counts_{1}_{2}.nii.gz'.format(avr_dir, roi_i, roi_j))
#         print('{0}/avr_bin_tract_counts_{1}_{2}.nii.gz'.format(avr_dir, roi_i, roi_j))

# Clear some memory
del V_min_avr
del V_bin_avr
del V
        
print('Done smoothing and saving.')

#print('\n== Done computing average tracts ==\n')


== Computing average tracts ==

Processing subject-wise tractography...
['A00008326', 'A00010893', 'A00013809', 'A00021039', 'A00023510', 'A00025566', 'A00027651', 'A00028150', 'A00028184', 'A00028185', 'A00028287', 'A00028340', 'A00028352', 'A00028389', 'A00028399', 'A00028429', 'A00028678', 'A00028753', 'A00028784', 'A00028842', 'A00028844', 'A00028845', 'A00028912', 'A00028994', 'A00029076', 'A00029092', 'A00029104', 'A00029215', 'A00029230', 'A00029231', 'A00030912', 'A00030913', 'A00030947', 'A00030989', 'A00031167', 'A00031216', 'A00031217', 'A00031459', 'A00031794', 'A00031871', 'A00032010', 'A00033011', 'A00033021', 'A00033232', 'A00033247', 'A00033248', 'A00033589', 'A00033609', 'A00033640', 'A00033647', 'A00033673', 'A00033735', 'A00033747', 'A00033748', 'A00033752', 'A00033764', 'A00033834', 'A00033849', 'A00034193', 'A00034827', 'A00034987', 'A00035036', 'A00035363', 'A00035437', 'A00035440', 'A00035504', 'A00035535', 'A00035562', 'A00035608', 'A00035625', 'A00035827', 'A0

  Processing subject: A00008326 (1 of 98)
  Processing subject: A00010893 (2 of 98)
  Processing subject: A00013809 (3 of 98)
  Processing subject: A00021039 (4 of 98)
  Processing subject: A00023510 (5 of 98)
  Processing subject: A00025566 (6 of 98)
  Processing subject: A00027651 (7 of 98)
  Processing subject: A00028150 (8 of 98)
  Processing subject: A00028184 (9 of 98)
  Processing subject: A00028185 (10 of 98)
  Processing subject: A00028287 (11 of 98)
  Processing subject: A00028340 (12 of 98)
  Processing subject: A00028352 (13 of 98)
  Processing subject: A00028389 (14 of 98)
  Processing subject: A00028399 (15 of 98)
  Processing subject: A00028429 (16 of 98)
  Processing subject: A00028678 (17 of 98)
  Processing subject: A00028753 (18 of 98)
  Processing subject: A00028784 (19 of 98)
  Processing subject: A00028842 (20 of 98)
  Processing subject: A00028844 (21 of 98)
  Processing subject: A00028845 (22 of 98)
  Processing subject: A00028912 (23 of 98)
  Processing subject

In [11]:
# Compute tract distances
#
# Computes the distance from a seed ROI along a tract to a target ROI.
# Uses a flood-fill approach to assign integers to voxels, representing
# the number of voxel steps required to reach that voxel from the seed ROI

print('\n== Computing tract distances ==\n')

# Create directories
params_gauss = params['gaussians']
sigma_axial = params_gauss['fwhm_axial'] / (2.0*math.sqrt(2.0*math.log(2.0)))
sigma_radial = params_gauss['fwhm_radial'] / (2.0*math.sqrt(2.0*math.log(2.0)))
threshold = params_gauss['threshold']
max_seg_length = params_gauss['max_seg_length']
gauss_max_radius = params_gauss['gauss_max_radius']
tract_thresh = params_gauss['tract_thresh']
seed_dilate = params_gauss['seed_dilate']

V_rois = {}
V_dists = {}

# Load ROI volumes
for roi in rois:
    roi_file = '{0}/{1}.{2}'.format(rois_dir, roi, roi_suffix)
    img = nib.load(roi_file)
    V_rois[roi] = np.squeeze(img.get_data())
    
if params_gen['clobber']:
    if os.path.isdir(dist_dir):
        shutil.rmtree(dist_dir)
    os.makedirs(dist_dir)
    
# Compute tract distances
rois_a = []
rois_b = []

# for a in range (0,len(rois)):
#     for b in range (a+1,len(rois)):
#         rois_a.append(rois[a])
#         rois_b.append(rois[b])
for roi_i in roi_pairs:
    for roi_j in roi_pairs[roi_i]:
        rois_a.append(roi_i)
        rois_b.append(roi_j)  

# del a,b 
c = 0
for c in tqdm_notebook(range (0,len(rois_a)), desc="Progress"):
    roi_a = rois_a[c]
    V_roi_a = V_rois[roi_a]
    roi_b = rois_b[c]
    V_roi_b = V_rois[roi_b]
    c += 1
        
    dist_img_ab = '{0}/tract_dist_{1}_{2}.nii.gz'.format(dist_dir, roi_a, roi_b)
    dist_img_ba = '{0}/tract_dist_{1}_{2}.nii.gz'.format(dist_dir, roi_b, roi_a)

    # Check if this has already been done and if so load it instead of computing it
    # (this can be redone if clobber=true)
    if not params_gen['clobber'] and os.path.isfile(dist_img_ab) and os.path.isfile(dist_img_ba):
        V_dists['{0}_{1}'.format(roi_a, roi_b)] = nib.load(dist_img_ab).get_data()
        V_dists['{0}_{1}'.format(roi_b, roi_a)] = nib.load(dist_img_ba).get_data()
        if params_gen['verbose']:
            print('  Images already exist for {0}/{1}; loading from file.'.format(roi_a, roi_b))
    else:

        tract_file = '{0}/avr_bin_tract_counts_{1}_{2}.nii.gz'.format(avr_dir, roi_a, roi_b)
        if not os.path.isfile(tract_file):
            tract_file = '{0}/avr_bin_tract_counts_{1}_{2}.nii.gz'.format(avr_dir, roi_b, roi_a)

        if not os.path.isfile(tract_file):
            if params_gen['verbose']:
                print('  **Warning: No file found for ROI pair {0}/{1}! Skipping...'.format(roi_a, roi_b))
        else:
            # Load and threshold average tract
            V_tract = nib.load(tract_file).get_data()
            
            V_mask = np.greater(V_tract, threshold).astype(np.float)
            V_tract = np.multiply(V_mask, V_tract)

            # Compute voxel steps from seed as distances
            V_dist_a = utils.get_tract_dist(V_mask, V_roi_a, seed_dilate, V_stop=V_roi_b)
            V_dist_b = utils.get_tract_dist(V_mask, V_roi_b, seed_dilate, V_stop=V_roi_a)
            V_dist = np.logical_and(V_dist_a>0, V_dist_b>0)

            if not np.any(V_dist):
                if params_gen['verbose']:
                    print('  ** Warning: Tracts do not overlap for {0}/{1}. Skipping'.format(roi_a, roi_b))
            else:

                 # Write distances to file
                img = nib.Nifti1Image(V_dist_a, V_img.affine, V_img.header)
                nib.save(img, dist_img_ab)
                img = nib.Nifti1Image(V_dist_b, V_img.affine, V_img.header)
                nib.save(img, dist_img_ba)

                if params_gen['verbose']:
                    print('  Done tract {0}/{1}'.format(roi_a, roi_b))
            
print('\n== Done computing tract distances ==\n')



== Computing tract distances ==



  Done tract BA6_left/BA6_right
  Done tract BA6_left/BA44_left
  Done tract BA6_left/BA44_right
  Done tract BA6_left/INS_left
  Done tract BA6_left/INS_right
  Done tract BA6_left/IPS_left
  ** Warning: Tracts do not overlap for BA6_left/IPS_right. Skipping
  Done tract BA6_right/BA44_left
  Done tract BA6_right/BA44_right
  Done tract BA6_right/INS_left
  ** Warning: Tracts do not overlap for BA6_right/INS_right. Skipping
  Done tract BA6_right/IPS_left
  Done tract BA6_right/IPS_right
  Done tract BA44_left/BA44_right
  Done tract BA44_left/INS_left
  ** Warning: Tracts do not overlap for BA44_left/INS_right. Skipping
  ** Warning: Tracts do not overlap for BA44_left/IPS_left. Skipping
  ** Warning: Tracts do not overlap for BA44_left/IPS_right. Skipping
  Done tract BA44_right/INS_left
  Done tract BA44_right/INS_right
  ** Warning: Tracts do not overlap for BA44_right/IPS_left. Skipping
  Done tract BA44_right/IPS_right
  ** Warning: Tracts do not overlap for INS_left/INS_right. 

In [10]:
print(V_tract.shape)
print(V_roi_b.shape)

(197, 233, 189)
(197, 233, 189, 1)


In [17]:
# Get center polylines and generate Gaussians
#
# The goal is to estimate the "core" trajectory of each tract between
# ROI pairs, if this is estimable. 
#
# This routine will also generate bidirectional tract estimates, 
# determined by the average value for each individual estimate


# Determine polylines representing center of average tract, if they
# exist. Apply a Gaussian weighting centered on these polylines.
# Multiply this with the original average tract.

# Debug:

import time

print('\n== Generating polylines and Gaussians ==\n')

params_gauss = params['gaussians']
sigma_axial = params_gauss['fwhm_axial'] / (2.0*math.sqrt(2.0*math.log(2.0)))
sigma_radial = params_gauss['fwhm_radial'] / (2.0*math.sqrt(2.0*math.log(2.0)))
threshold = params_gauss['threshold']
max_seg_length = params_gauss['max_seg_length']
gauss_max_radius = params_gauss['gauss_max_radius']
tract_thresh = params_gauss['tract_thresh']
mask_dilate = params_gauss['mask_dilate']

if params_gen['clobber']:
    if os.path.isdir(gauss_dir):
        shutil.rmtree(gauss_dir)
    os.makedirs(gauss_dir)
    if os.path.isdir(gauss_fail_dir):
        shutil.rmtree(gauss_fail_dir)
    os.makedirs(gauss_fail_dir)
    if os.path.isdir(final_dir):
        shutil.rmtree(final_dir)
    os.makedirs(final_dir)
    if os.path.isdir(final_fail_dir):
        shutil.rmtree(final_fail_dir)
    os.makedirs(final_fail_dir)

# Load ROI volumes

# Read ROI list
rois = []
with open(params_gen['rois_list'],'r') as roi_file:
    reader = csv.reader(roi_file)
    for row in reader:
        rois.append(row[0])

V_rois = {}
for roi in rois:
    roi_file = '{0}/{1}.{2}'.format(rois_dir, roi, roi_suffix)
    img = nib.load(roi_file)
    V_rois[roi] = img.get_data()
    
tract_failed = {}

ts = time.gmtime()
timestamp = time.strftime("%Y-%m-%d %H:%M:%S", ts)

fail_log = '{0}/failed'.format(log_dir)
with open(fail_log, 'a') as logfile:
    logfile.write('\n\n--New run: {0}\n'.format(timestamp))
        
success_log = '{0}/success'.format(log_dir)
with open(success_log, 'a') as logfile:
    logfile.write('\n\n--New run: {0}\n'.format(timestamp))
    
violations_log = '{0}/violations'.format(log_dir)
with open(violations_log, 'a') as logfile:
    logfile.write('\n\n--New run: {0}\n'.format(timestamp))

N_roi = len(rois)

tract_list = []
rois_a = []
rois_b = []

for roi_i in roi_pairs:
    for roi_j in roi_pairs[roi_i]:
        tract_name = '{0}_{1}'.format(roi_i,roi_j)
        tract_list.append(tract_name)
        rois_a.append(roi_i)
        rois_b.append(roi_j)  


# for roi_a in rois:
#     for roi_b in [x for x in rois if x != roi_a]:
#         tract_name = '{0}_{1}'.format(roi_a,roi_b)
#         tract_list.append(tract_name)
#         rois_a.append(roi_a)
#         rois_b.append(roi_b)

c = 0

for tract_name in tqdm_notebook(tract_list, desc='Progress'):
    
    roi_a = rois_a[c]
    roi_b = rois_b[c]
    c += 1

    # Target mask is both ROIs
    V_target_a = np.squeeze(V_rois[roi_a])
    V_target_b = np.squeeze(V_rois[roi_b])
    tract_failed[tract_name] = False

    dist_file = '{0}/tract_dist_{1}_{2}.nii.gz'.format(dist_dir, roi_a, roi_b)

    
    tract_file = '{0}/avr_min_tract_counts_{1}_{2}.nii.gz'.format(avr_dir, roi_a, roi_b)
    if not os.path.isfile(tract_file):
        tract_file = '{0}/avr_min_tract_counts_{1}_{2}.nii.gz'.format(avr_dir, roi_b, roi_a)

    if not os.path.isfile(dist_file):
        if params_gen['verbose']:
            print('  **Warning: No distances file found for ROI pair {0}/{1}! Skipping...'.format(roi_a, roi_b))
        tract_failed[tract_name] = True
        # Log failure
        with open(fail_log,'a') as logfile:
            logfile.write('{0}|{1},File not found\n'.format(roi_a,roi_b))
    else:

        if (not params_gen['clobber'] and
            os.path.isfile('{0}/tract_final_{1}.nii.gz'.format(final_dir, tract_name))):
            if params_gen['verbose']:
                print('  Output for {0} already exists. Skipping this tract.'.format(tract_name))

        else:

            # Dilate A and B to use as stop masks
            if mask_dilate:
                V_target_a = utils.dilate_mask(np.greater(V_target_a,0))
                V_target_b = utils.dilate_mask(np.greater(V_target_b,0))

            if params_gen['debug']:
                # Write to debug file
                img = nib.Nifti1Image(V_target_a, V_img.affine, V_img.header)
                nib.save(img, '{0}/dilated_target_a_{1}.nii.gz'.format(debug_dir, tract_name))
                img = nib.Nifti1Image(V_target_b, V_img.affine, V_img.header)
                nib.save(img, '{0}/dilated_target_b_{1}.nii.gz'.format(debug_dir, tract_name))

            V_tract = nib.load(tract_file).get_data()
            V_orig = V_tract.copy()
            # Threshold
            V_tract[np.less(V_tract, threshold)] = 0

            V_dist = np.round(nib.load(dist_file).get_data())

            if V_dist is None:
                tract_failed[tract_name] = True
                if params_gen['verbose']:
                    print('  **Warning: Skipping {0}; tract not found.'.format(tract_name))
                # Log failure
                with open(fail_log,'a') as logfile:
                    logfile.write('{0},Tract not found\n'.format(tract_name))
            else:
                dist_max = int(V_dist.max())
                if params_gen['debug']:
                    print('  Tract {0}: max_dist={1}'.format(tract_name, dist_max))

                # Compute polylines
                maxes = np.array([])
                constraints_failed = 0
                found_target_a = False
                found_target_b = False
                
                # Start in the middle to avoid endpoint madness
                d_start = round(dist_max/2)
                
                maxes = np.zeros((dist_max,3), dtype=int)

                for k in tqdm_notebook(range(0, d_start), desc='Distances'.format(c)):

                    for oe in [0,1]:
                    
                        if oe == 0:
                            if found_target_a:
                                d = -1
                            else:
                                d = d_start - k
                        else:
                            if found_target_b:
                                d = -1
                            else:
                                d = d_start + k - 1
                            
                        if d >= 0 and d <= dist_max:
                            
                            T_idx = np.equal(V_dist,d)
                            T = V_tract[T_idx]
                            T_idx = np.flatnonzero(T_idx)
                            sidx = T.argsort()[::-1]
                            idx_max = np.unravel_index(T_idx[sidx[0]], V_tract.shape)
                            
                            if k == 0:
                                maxes[d, :] = idx_max
                            else:
                                if oe == 0:
                                    idx_prev = maxes[d+1,:]
                                else:
                                    idx_prev = maxes[d-1,:]

                                # Find maximal value which satisfies length constraint
                                l = 0
                                is_valid = False
                                while not is_valid and l < len(T):
                                    idx_l = np.unravel_index(T_idx[sidx[l]], V_tract.shape)
                                    seg_length = math.sqrt((idx_l[0]-idx_prev[0])**2 + \
                                                           (idx_l[1]-idx_prev[1])**2 + \
                                                           (idx_l[2]-idx_prev[2])**2)
                                    if seg_length < max_seg_length:
                                        maxes[d,:] = idx_l
                                        is_valid = True
                                    l+=1
                                if not is_valid:
                                    # Failed to meet constraint; add anyway but note failure
                                    constraints_failed += 1
                                    if params_gen['debug']:
                                        print('   * Debug: Constraint fail: len ({0}) > max_len({1})'. \
                                              format(seg_length, max_seg_length))
                                    
                            # Is this voxel adjacent to target? Then ignore any further distances
                            if not found_target_a and np.any(np.logical_and(np.equal(V_dist,d),np.greater(V_target_a,0))):
                                if params_gen['verbose']:
                                    print('  Tract {0}: Found target A @ d={1}.'.format(tract_name, d))
                                found_target_a = True
                        
                            if not found_target_b and np.any(np.logical_and(np.equal(V_dist,d),np.greater(V_target_b,0))):
                                if params_gen['verbose']:
                                    print('  Tract {0}: Found target B @ d={1}.'.format(tract_name, d))
                                found_target_b = True
                        
                            if found_target_a and found_target_b:
                                break
                        
                        # If target was found, we're done
                        if found_target_a and found_target_b:
                            break

                found_target = found_target_a and found_target_b
                            
                # Only keep non-zero rows
                maxes = maxes[~np.all(maxes == 0, axis=1)]
                            
                my_target_gauss = gauss_fail_dir
                my_target_final = final_fail_dir

                if constraints_failed > 0:
                    if params_gen['verbose']:
                        print('  Tract {0}: Polyline had {1} constraint violation(s).' \
                                  .format(tract_name, constraints_failed))
                    with open(violations_log, 'a') as logfile:
                        logfile.write('{0},{1}\n'.format(tract_name, constraints_failed))

                    if params_gauss['fail_constraints']:
                        tract_failed[tract_name] = True
                    else:
                        my_target_gauss = gauss_dir
                        my_target_final = final_dir

                elif not found_target:
                    if params_gen['verbose']:
                        print('  Tract {0}: Did not find target ROI.'.format(tract_name))
                    tract_failed[tract_name] = True

                    with open(fail_log, 'a') as logfile:
                        logfile.write('{0},{1}'.format(tract_name, dist_max))
                else:
                    my_target_gauss = gauss_dir
                    my_target_final = final_dir
                    with open(success_log, 'a') as logfile:
                        logfile.write('{0},{1}\n'.format(tract_name, dist_max))

                if maxes.size == 0:
                    print('Maxes not found for tract {0}!? Failing this tract.'.format(tract_name))
                    tract_failed[tract_name] = True
                    with open(fail_log, 'a') as logfile:
                        logfile.write('{0},{1}\n'.format(tract_name, dist_max))
                else:
                        
                    # Smooth the polyline defined by maxes at each distance
                    maxes_vox = np.round(utils.smooth_polyline_ma(maxes, 3))

                    # Transform from voxel to world coordinates and smooth
                    maxes = utils.smooth_polyline_ma(utils.voxel_to_world(maxes, V_img.header))

                    # Write polyline to Mgui format
                    utils.write_polyline_mgui(maxes, '{0}/maxes_{1}_sm3.poly3d'.format(polyline_dir, tract_name), tract_name)

                    # For each voxel, derive Gaussian for neighbours within gauss_max_radius
                    V_gauss = np.zeros(V_tract.shape)
                    max_radius = round(gauss_max_radius)
                    nv = maxes_vox.shape[0]

                    for v in tqdm_notebook(range(1, nv),desc='Gaussians'):
                        p = maxes_vox[v,:]
                        p0 = maxes_vox[v-1,:]
                        v_axis = p - p0

                        v_len = np.linalg.norm(v_axis)
                        if v_len == 0:
                            if params_gen['debug']:
                                print('Len @ {0} is zero! p_1={1} | p_2={2}'.format(v,p,p0))
                        else:
                            v_axis = np.divide(v_axis, np.linalg.norm(v_axis)) # Unit vector

                            for i in range(max(0, p[0]-max_radius), min(V_gauss.shape[0]-1, p[0]+max_radius)):
                                for j in range(max(0, p[1]-max_radius), min(V_gauss.shape[1]-1, p[1]+max_radius)):
                                    for k in range(max(0, p[2]-max_radius), min(V_gauss.shape[2]-1, p[2]+max_radius)):
                                        vox_ijk = [i,j,k]
                                        v_d = np.subtract(vox_ijk, p)
                                        v_ax = abs(np.dot(v_d, v_axis))                     # Magnitude of axial component
                                        v_rad = math.sqrt(v_ax**2 + np.linalg.norm(v_d)**2) # Magnitude of axial component

                                        # Compute Gaussian, set voxel value to maximal Gaussian encountered
                                        gauss = stats.norm.pdf(v_ax, 0, sigma_axial) * stats.norm.pdf(v_rad, 0, sigma_radial)
                                        V_gauss[i,j,k] = max(gauss, V_gauss[i,j,k])

                    # Apply a bit of smoothing
                    V_gauss = utils.smooth_volume(V_gauss, V_img, 5)

                    # Multiply with original (unthresholded) tract
                    V_tract = np.multiply(V_orig, V_gauss)
                    V_tract = np.multiply(V_tract, np.greater(V_tract, tract_thresh))
                    V_tract = np.divide(V_tract, V_tract.max())
                    V_dist = np.round(np.multiply(V_dist, np.greater(V_tract, 0)))
                    dist_max = int(np.max(V_dist))

                    # Write results to NIFTI image files
                    img = nib.Nifti1Image(V_gauss, V_img.affine, V_img.header)
                    nib.save(img, '{0}/gauss_max_{1}.nii.gz'.format(my_target_gauss, tract_name))

                    img = nib.Nifti1Image(V_tract, V_img.affine, V_img.header)
                    nib.save(img, '{0}/tract_final_{1}.nii.gz'.format(my_target_final, tract_name))

                    # Normalize tract at each distance
                    for d in range(1, dist_max):
                        idx = np.nonzero(np.equal(V_dist,d))
                        if np.any(idx):
                            V = V_tract[idx]
                            V = np.divide(V, V.max())
                            V_tract[idx] = V

                    img = nib.Nifti1Image(V_tract, V_img.affine, V_img.header)
                    nib.save(img, '{0}/tract_final_norm_{1}.nii.gz'.format(my_target_final, tract_name))

                    if params_gen['verbose']:
                        print('  Computed distances, polylines, and Gaussians for {0}/{1}'.format(roi_a, roi_b))
    
print('\n== Done generating polylines and Gaussians ==\n')



== Generating polylines and Gaussians ==



  Tract BA6_left_BA6_right: max_dist=10


  Tract BA6_left_BA6_right: Found target B @ d=5.
  Tract BA6_left_BA6_right: Found target A @ d=1.


  Computed distances, polylines, and Gaussians for BA6_left/BA6_right
  Tract BA6_left_BA44_left: max_dist=41


  Tract BA6_left_BA44_left: Found target B @ d=35.
  Tract BA6_left_BA44_left: Found target A @ d=1.


...

Len @ 1 is zero! p_1=[137 141 104] | p_2=[137 141 104]
  Computed distances, polylines, and Gaussians for BA44_right/INS_left
  Tract BA44_right_INS_right: max_dist=45


   * Debug: Constraint fail: len (14.730919862656235) > max_len(4)
   * Debug: Constraint fail: len (228.69411885748178) > max_len(4)
   * Debug: Constraint fail: len (228.08770243044668) > max_len(4)
   * Debug: Constraint fail: len (228.48632344190756) > max_len(4)
   * Debug: Constraint fail: len (203.0615670184784) > max_len(4)
   * Debug: Constraint fail: len (202.5561650505854) > max_len(4)
   * Debug: Constraint fail: len (202.05444810743464) > max_len(4)
   * Debug: Constraint fail: len (201.55644370746373) > max_len(4)
   * Debug: Constraint fail: len (201.06217943710845) > max_len(4)
   * Debug: Constraint fail: len (200.049993751562) > max_len(4)
   * Debug: Constraint fail: len (199.04270898478046) > max_len(4)
   * Debug: Constraint fail: len (198.5572965166478) > max_len(4)
   * Debug: Constraint fail: len (198.07574308834486) > max_len(4)
   * Debug: Constraint fail: len (197.0786644972002) > max_len(4)
   * Debug: Constraint fail: len (196.60366222428308) > max_len(4)
 

  Computed distances, polylines, and Gaussians for BA44_right/INS_right
  **Warning: No distances file found for ROI pair BA44_right/IPS_left! Skipping...
  Tract BA44_right_IPS_right: max_dist=43


  Tract BA44_right_IPS_right: Found target B @ d=37.
  Tract BA44_right_IPS_right: Found target A @ d=1.


  Computed distances, polylines, and Gaussians for BA44_right/IPS_right
  **Warning: No distances file found for ROI pair INS_left/INS_right! Skipping...
  **Warning: No distances file found for ROI pair INS_left/IPS_left! Skipping...
  **Warning: No distances file found for ROI pair INS_left/IPS_right! Skipping...
  **Warning: No distances file found for ROI pair INS_right/IPS_left! Skipping...
  **Warning: No distances file found for ROI pair INS_right/IPS_right! Skipping...
  **Warning: No distances file found for ROI pair IPS_left/IPS_right! Skipping...

== Done generating polylines and Gaussians ==



In [18]:
# Finally, average tracts across both directions
print('\n== Computing bidirectional tracts...')

params_gauss = params['gaussians']
sigma_axial = params_gauss['fwhm_axial'] / (2.0*math.sqrt(2.0*math.log(2.0)))
sigma_radial = params_gauss['fwhm_radial'] / (2.0*math.sqrt(2.0*math.log(2.0)))
threshold = params_gauss['threshold']
max_seg_length = params_gauss['max_seg_length']
gauss_max_radius = params_gauss['gauss_max_radius']
tract_thresh = params_gauss['tract_thresh']

# Read ROI list
rois = []
with open(params_gen['rois_list'],'r') as roi_file:
    reader = csv.reader(roi_file)
    for row in reader:
        rois.append(row[0])
        
V_rois = {}
for roi in rois:
    roi_file = '{0}/{1}.{2}'.format(rois_dir, roi, roi_suffix)
    img = nib.load(roi_file)
    V_rois[roi] = img.get_data()

N_roi = len(rois)

tract_list = []
rois_a = []
rois_b = []

for roi_a in rois:
    for roi_b in [x for x in rois if x != roi_a]:
        tract_name = '{0}_{1}'.format(roi_a,roi_b)
        tract_list.append(tract_name)
        rois_a.append(roi_a)
        rois_b.append(roi_b)

for a in range(0, len(rois)-1):
    roi_a = rois[a]
    for b in range(a+1, len(rois)):
        roi_b = rois[b]
        ab = '{0}_{1}'.format(roi_a,roi_b)
        ba = '{0}_{1}'.format(roi_b,roi_a)
        
        tract_failed_ab = not os.path.isfile('{0}/tract_final_{1}.nii.gz'.format(final_dir, ab))
        tract_failed_ba = not os.path.isfile('{0}/tract_final_{1}.nii.gz'.format(final_dir, ba))
        
        if not (tract_failed_ab and tract_failed_ba):
            
            # Unnormalized version
            if tract_failed_ab:
                V = nib.load('{0}/tract_final_{1}.nii.gz'.format(final_dir, ba)).get_data()
            elif tract_failed_ba:
                V = nib.load('{0}/tract_final_{1}.nii.gz'.format(final_dir, ab)).get_data()
            else:
                V_ab = nib.load('{0}/tract_final_{1}.nii.gz'.format(final_dir, ab)).get_data()
                V_ba = nib.load('{0}/tract_final_{1}.nii.gz'.format(final_dir, ba)).get_data()
                V = np.max( np.array([ V_ab, V_ba ]), axis=0 )
            
            V_blobs = utils.label_blobs(V, threshold)
            if len(np.unique(V_blobs)) > 2:
                V_blobs = utils.retain_adjacent_blobs(V_blobs, [V_rois[roi_a], V_rois[roi_b]])
                if len(np.unique(V_blobs)) > 2:
                    print('  * Tract has multiple tract segments (unfixed): {0}|{1}'.format(roi_a, roi_b))
                else:
                    print('  * Tract had multiple tract segments (1 retained): {0}|{1}'.format(roi_a, roi_b))
                V = np.multiply(V, V_blobs>0)
            
            img = nib.Nifti1Image(V, V_img.affine, V_img.header)
            nib.save(img, '{0}/tract_final_bidir_{1}.nii.gz'.format(final_dir, ab))

            # Normalized version
            if tract_failed_ab:
                V = nib.load('{0}/tract_final_norm_{1}.nii.gz'.format(final_dir, ba)).get_data()
            elif tract_failed_ba:
                V = nib.load('{0}/tract_final_norm_{1}.nii.gz'.format(final_dir, ab)).get_data()
            else:
                V_ab = nib.load('{0}/tract_final_norm_{1}.nii.gz'.format(final_dir, ab)).get_data()
                V_ba = nib.load('{0}/tract_final_norm_{1}.nii.gz'.format(final_dir, ba)).get_data()
                V = V_ab + V_ba  #np.max( np.array([ V_ab, V_ba ]), axis=0 )
                V[np.logical_and(V_ab,V_ba)] = V[np.logical_and(V_ab,V_ba)] / 2
            
            V = np.multiply(V, V_blobs>0)
            
            # Check for blobs that don't connect ROIs; remove these if found
            img = nib.Nifti1Image(V, V_img.affine, V_img.header)
            nib.save(img, '{0}/tract_final_norm_bidir_{1}.nii.gz'.format(final_dir, ab))
            
            if params_gen['verbose']:
                print('  Wrote average bidirectional tract for {0}/{1}'.format(roi_a, roi_b))
                
print('\n== Done generating bidirectional tracts ==\n')


== Computing bidirectional tracts...
  Wrote average bidirectional tract for BA6_left/BA6_right
  Wrote average bidirectional tract for BA6_left/BA44_left
  Wrote average bidirectional tract for BA6_left/BA44_right
  Wrote average bidirectional tract for BA6_left/INS_left
  Wrote average bidirectional tract for BA6_left/INS_right
  Wrote average bidirectional tract for BA6_left/IPS_left
  Wrote average bidirectional tract for BA6_right/BA44_left
  Wrote average bidirectional tract for BA6_right/BA44_right
  Wrote average bidirectional tract for BA6_right/INS_left
  Wrote average bidirectional tract for BA6_right/IPS_left
  Wrote average bidirectional tract for BA6_right/IPS_right
  Wrote average bidirectional tract for BA44_left/BA44_right
  Wrote average bidirectional tract for BA44_left/INS_left
  Wrote average bidirectional tract for BA44_right/INS_left
  Wrote average bidirectional tract for BA44_right/INS_right
  Wrote average bidirectional tract for BA44_right/IPS_right

== Done

In [19]:
# Compute average orientations across subjects
#
# For each tract that was successfully identified, we will now use the
# average voxel-wise streamline orientation information generated by ProbtrackX 
# to compute the average orientation across subjects, for each voxel in the tract.
# This will later be used to compute how strongly diffusion loads on the average
# orientation for each subject, which can be regressed against factors of interest.

print('\n== Computing average streamline orientations ==\n')

params_avdir = params['average_directions']
tract_thresh = params_avdir['threshold']

if params_gen['debug']:
    last_clobber = params_gen['clobber']
    params_gen['clobber'] = True

# Make sure the tract output exists

final_dir = os.path.join(tracts_dir, 'final')
if not os.path.isdir(final_dir):
    print('  No tracts appear to have been generated. Stopping.')
    raise
    
avdir_dir = os.path.join(tracts_dir, 'avrdir')
if not os.path.isdir(avdir_dir):
    os.makedirs(avdir_dir)

subjects = []
with open('{0}/subjects_avr_{1}.list'.format(tracts_dir, params_gen['network_name'])) as subj_file:
    reader = csv.reader(subj_file)
    for row in reader:
        subjects.append(row[0])
        
subjects = sorted(subjects)

tract_list = []
rois_a = []
rois_b = []

for a in range(0, len(rois)-1):
    roi_a = rois[a]
    for b in range(a+1, len(rois)):
        roi_b = rois[b]
        tract_name = '{0}_{1}'.format(roi_a,roi_b)
        tract_list.append(tract_name)
        rois_a.append(roi_a)
        rois_b.append(roi_b)

for c in tqdm_notebook(range(0, len(rois_a)), desc='Total Progress'):
    roi_a = rois_a[c]
    roi_b = rois_b[c]
    tract_name = '{0}_{1}'.format(roi_a,roi_b)
    output_file = '{0}/avrdir_{1}.nii.gz'.format(avdir_dir, tract_name)
    
    if not params_gen['clobber'] and os.path.isfile(output_file):
        print('  Average orientations already computed for {0}. Skipping.'.format(tract_name))
    else:
        tract_file = '{0}/tract_final_bidir_{1}.nii.gz'.format(final_dir, tract_name)

        if not os.path.isfile(tract_file):
            print('  No tract found for {0}. Skipping.'.format(tract_name))
        else:
            img = nib.load(tract_file)
            V_tract = img.get_data()

            s = (3,) + V_tract.shape
            ss = V_tract.shape + (3,)
            V_avdir = np.zeros(ss)
            V_denom = np.zeros(ss)

            V_img3 = None

            # For each subject, read local direction vectors
            for subject in tqdm_notebook(subjects, desc='{0}:'.format(tract_name)):
                prefix_sub = '{0}{1}'.format(params_gen['prefix'], subject)
                
                # AB orientations
                subject_dir = os.path.join(project_dir, params_gen['deriv_dir'], prefix_sub, \
                                           params_gen['sub_dirs'], params_gen['dwi_dir'], \
                                           'probtrackX', params_gen['network_name'], roi_a)
                avdir_file = '{0}/target_localdir_{1}.nii.gz'.format(subject_dir, roi_b)

                V_img3 = nib.load(avdir_file)
                V_dir = V_img3.get_data()
                V_dir[np.less(V_tract, tract_thresh)] = 0

                # Ensure the maximal vector component is positive (we are concerned with
                # orientation rather than directions); flip those that aren't
                idx = np.nonzero(V_dir)
                V_denom[idx] = np.add(V_denom[idx],1)
                V_dir = utils.make_vectors_positive( V_dir )
                
                V_avdir = np.add(V_avdir, V_dir)
                
                # BA orientations
                subject_dir = os.path.join(project_dir, params_gen['deriv_dir'], prefix_sub, \
                                           params_gen['sub_dirs'], params_gen['dwi_dir'], \
                                           'probtrackX', params_gen['network_name'], roi_b)
                avdir_file = '{0}/target_localdir_{1}.nii.gz'.format(subject_dir, roi_a)

                V_img3 = nib.load(avdir_file)
                V_dir = V_img3.get_data()
                V_dir[np.less(V_tract, tract_thresh)] = 0
                
                # Ensure the maximal vector component is positive (we are concerned with
                # orientation rather than directions); flip those that aren't
                idx = np.nonzero(V_dir)
                V_denom[idx] = np.add(V_denom[idx],1)
                V_dir = utils.make_vectors_positive( V_dir )
                V_avdir = np.add(V_avdir, V_dir)
                
            # Divide to get average, and mask to exclude to this tract
            idx = np.equal(V_denom,0)
            V_denom[idx] = 1
            V_avdir = np.divide(V_avdir, V_denom)

            nidx = np.transpose(np.nonzero(V_avdir))
            
            # Normalize results
#             for i in range(0, nidx.shape[0]):
#                 x = nidx[i,0]
#                 y = nidx[i,1]
#                 z = nidx[i,2]
#                 v = np.squeeze(V_avdir[x, y, z, :])
#                 nm = np.linalg.norm(v)
#                 if nm > 0:
#                     v = np.divide(v, nm)
#                     V_avdir[x,y,z,:] = v

            # Write average directions to file
            img = nib.Nifti1Image(V_avdir, V_img3.affine, V_img3.header)
            nib.save(img, output_file)

            print('  Computed average orientations for {0}'.format(tract_name))
            
if params_gen['debug']:
    params_gen['clobber'] = last_clobber
            
print('\n== Done computing average streamline orientations ==\n')
        


== Computing average streamline orientations ==



  Computed average orientations for BA6_left_BA6_right


  Computed average orientations for BA6_left_BA44_left


  Computed average orientations for BA6_left_BA44_right


  Computed average orientations for BA6_left_INS_left


...

  Computed average orientations for BA44_left_BA44_right


  Computed average orientations for BA44_left_INS_left
  No tract found for BA44_left_INS_right. Skipping.
  No tract found for BA44_left_IPS_left. Skipping.
  No tract found for BA44_left_IPS_right. Skipping.


  Computed average orientations for BA44_right_INS_left


  Computed average orientations for BA44_right_INS_right
  No tract found for BA44_right_IPS_left. Skipping.


  Computed average orientations for BA44_right_IPS_right
  No tract found for INS_left_INS_right. Skipping.
  No tract found for INS_left_IPS_left. Skipping.
  No tract found for INS_left_IPS_right. Skipping.
  No tract found for INS_right_IPS_left. Skipping.
  No tract found for INS_right_IPS_right. Skipping.
  No tract found for IPS_left_IPS_right. Skipping.

== Done computing average streamline orientations ==



In [20]:
# Perform regressions in DWI space
#
# Uses the average orientation computed above to obtain tract-specific measures of
# diffusion for each voxel. Voxel-wise regressions are performed to determine how strongly
# the diffusion profile in each voxel loads (in terms of beta cofficients) onto the average 
# orientation for a given tract. These beta cofficients can subsequently be used for statistical
# analysis with subject-wise factors of interest (e.g., age, cognitive performance, clinical
# status, etc.)
#

import dwitracts

params_regress = params['dwi_regressions']
tmp_dir = params_gen['temp_dir']

print('== Computing DWI regressions ==')

# Make temp dir, empty it if it already exists
if os.path.isdir(tmp_dir):
    shutil.rmtree(tmp_dir)
os.makedirs(tmp_dir)

# Ensure input exists
if not os.path.isdir(tracts_dir):
    print('  *Error: No directory found: {0}. Stopping.'.format(tracts_dir))
    raise
    
if not os.path.isdir(project_dir):
    print('  *Error: No directory found: {0}. Stopping.'.format(project_dir))
    raise

if not os.path.isdir(rois_dir):
    print('  *Error: No directory found: {0}. Stopping.'.format(rois_dir))
    raise

# Create output directory
output_dir = '{0}/{1}'.format(tracts_dir, params_regress['regress_dir'])
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

# Read and sort subject list
subjects = []
with open('{0}/subjects_avr_{1}.list'.format(tracts_dir, params_gen['network_name'])) as subj_file:
    reader = csv.reader(subj_file)
    for row in reader:
        subjects.append(row[0])
subjects = sorted(subjects)

for subject in tqdm_notebook(subjects, desc='Progress: '):
    
    if params_gen['verbose']:
        print('Starting processing of subject {0}.'.format(subject))
    
    failed = dwitracts.process_regression_dwi(subject, params)
    
    if failed == 0:
        print('Successfully processed subject {0}.'.format(subject))
    else:
        print('Error when processing subject {0}.'.format(subject))


== Computing DWI regressions ==


Starting processing of subject A00008326.
  - Tract IPL_L_gwfa_IPL_R_gwfa not found. Skipping.
  - Tract IPL_L_gwfa_IPS_L_gwfa not found. Skipping.
  - Tract IPL_L_gwfa_IPS_R_gwfa not found. Skipping.
  - Tract IPL_L_gwfa_SPL_L_gwfa not found. Skipping.
  - Tract IPL_L_gwfa_SPL_R_gwfa not found. Skipping.
  - Tract IPL_L_gwfa_dPMC_L_gwfa not found. Skipping.
  - Tract IPL_L_gwfa_dPMC_R_gwfa not found. Skipping.
  - Tract IPL_L_gwfa_vPMC_L_gwfa not found. Skipping.
  - Tract IPL_L_gwfa_vPMC_R_gwfa not found. Skipping.
  - Tract IPL_R_gwfa_IPS_L_gwfa not found. Skipping.
  - Tract IPL_R_gwfa_IPS_R_gwfa not found. Skipping.
  - Tract IPL_R_gwfa_SPL_L_gwfa not found. Skipping.
  - Tract IPL_R_gwfa_SPL_R_gwfa not found. Skipping.
  - Tract IPL_R_gwfa_dPMC_L_gwfa not found. Skipping.
  - Tract IPL_R_gwfa_dPMC_R_gwfa not found. Skipping.
  - Tract IPL_R_gwfa_vPMC_L_gwfa not found. Skipping.
  - Tract IPL_R_gwfa_vPMC_R_gwfa not found. Skipping.
  - Tract IPS_L_gwfa_IPS_R_gwfa not found. Skippi

  - Tract IPL_L_gwfa_IPL_R_gwfa not found. Skipping.
  - Tract IPL_L_gwfa_IPS_L_gwfa not found. Skipping.
  - Tract IPL_L_gwfa_IPS_R_gwfa not found. Skipping.
  - Tract IPL_L_gwfa_SPL_L_gwfa not found. Skipping.
  - Tract IPL_L_gwfa_SPL_R_gwfa not found. Skipping.
  - Tract IPL_L_gwfa_dPMC_L_gwfa not found. Skipping.
  - Tract IPL_L_gwfa_dPMC_R_gwfa not found. Skipping.
  - Tract IPL_L_gwfa_vPMC_L_gwfa not found. Skipping.
  - Tract IPL_L_gwfa_vPMC_R_gwfa not found. Skipping.
  - Tract IPL_R_gwfa_IPS_L_gwfa not found. Skipping.
  - Tract IPL_R_gwfa_IPS_R_gwfa not found. Skipping.
  - Tract IPL_R_gwfa_SPL_L_gwfa not found. Skipping.
  - Tract IPL_R_gwfa_SPL_R_gwfa not found. Skipping.
  - Tract IPL_R_gwfa_dPMC_L_gwfa not found. Skipping.
  - Tract IPL_R_gwfa_dPMC_R_gwfa not found. Skipping.
  - Tract IPL_R_gwfa_vPMC_L_gwfa not found. Skipping.
  - Tract IPL_R_gwfa_vPMC_R_gwfa not found. Skipping.
  - Tract IPS_L_gwfa_IPS_R_gwfa not found. Skipping.
  - Tract IPS_L_gwfa_SPL_L_gwfa not fo

  - Tract IPL_L_gwfa_IPL_R_gwfa not found. Skipping.
  - Tract IPL_L_gwfa_IPS_L_gwfa not found. Skipping.
  - Tract IPL_L_gwfa_IPS_R_gwfa not found. Skipping.
  - Tract IPL_L_gwfa_SPL_L_gwfa not found. Skipping.
  - Tract IPL_L_gwfa_SPL_R_gwfa not found. Skipping.
  - Tract IPL_L_gwfa_dPMC_L_gwfa not found. Skipping.
  - Tract IPL_L_gwfa_dPMC_R_gwfa not found. Skipping.
  - Tract IPL_L_gwfa_vPMC_L_gwfa not found. Skipping.
  - Tract IPL_L_gwfa_vPMC_R_gwfa not found. Skipping.
  - Tract IPL_R_gwfa_IPS_L_gwfa not found. Skipping.
  - Tract IPL_R_gwfa_IPS_R_gwfa not found. Skipping.
  - Tract IPL_R_gwfa_SPL_L_gwfa not found. Skipping.
  - Tract IPL_R_gwfa_SPL_R_gwfa not found. Skipping.
  - Tract IPL_R_gwfa_dPMC_L_gwfa not found. Skipping.
  - Tract IPL_R_gwfa_dPMC_R_gwfa not found. Skipping.
  - Tract IPL_R_gwfa_vPMC_L_gwfa not found. Skipping.
  - Tract IPL_R_gwfa_vPMC_R_gwfa not found. Skipping.
  - Tract IPS_L_gwfa_IPS_R_gwfa not found. Skipping.
  - Tract IPS_L_gwfa_SPL_L_gwfa not fo

  - Tract IPL_L_gwfa_IPL_R_gwfa not found. Skipping.
  - Tract IPL_L_gwfa_IPS_L_gwfa not found. Skipping.
  - Tract IPL_L_gwfa_IPS_R_gwfa not found. Skipping.
  - Tract IPL_L_gwfa_SPL_L_gwfa not found. Skipping.
  - Tract IPL_L_gwfa_SPL_R_gwfa not found. Skipping.
  - Tract IPL_L_gwfa_dPMC_L_gwfa not found. Skipping.
  - Tract IPL_L_gwfa_dPMC_R_gwfa not found. Skipping.
  - Tract IPL_L_gwfa_vPMC_L_gwfa not found. Skipping.
  - Tract IPL_L_gwfa_vPMC_R_gwfa not found. Skipping.
  - Tract IPL_R_gwfa_IPS_L_gwfa not found. Skipping.
  - Tract IPL_R_gwfa_IPS_R_gwfa not found. Skipping.
  - Tract IPL_R_gwfa_SPL_L_gwfa not found. Skipping.
  - Tract IPL_R_gwfa_SPL_R_gwfa not found. Skipping.
  - Tract IPL_R_gwfa_dPMC_L_gwfa not found. Skipping.
  - Tract IPL_R_gwfa_dPMC_R_gwfa not found. Skipping.
  - Tract IPL_R_gwfa_vPMC_L_gwfa not found. Skipping.
  - Tract IPL_R_gwfa_vPMC_R_gwfa not found. Skipping.
  - Tract IPS_L_gwfa_IPS_R_gwfa not found. Skipping.
  - Tract IPS_L_gwfa_SPL_L_gwfa not fo

...

  - Tract IPL_L_gwfa_IPL_R_gwfa not found. Skipping.
  - Tract IPL_L_gwfa_IPS_L_gwfa not found. Skipping.
  - Tract IPL_L_gwfa_IPS_R_gwfa not found. Skipping.
  - Tract IPL_L_gwfa_SPL_L_gwfa not found. Skipping.
  - Tract IPL_L_gwfa_SPL_R_gwfa not found. Skipping.
  - Tract IPL_L_gwfa_dPMC_L_gwfa not found. Skipping.
  - Tract IPL_L_gwfa_dPMC_R_gwfa not found. Skipping.
  - Tract IPL_L_gwfa_vPMC_L_gwfa not found. Skipping.
  - Tract IPL_L_gwfa_vPMC_R_gwfa not found. Skipping.
  - Tract IPL_R_gwfa_IPS_L_gwfa not found. Skipping.
  - Tract IPL_R_gwfa_IPS_R_gwfa not found. Skipping.
  - Tract IPL_R_gwfa_SPL_L_gwfa not found. Skipping.
  - Tract IPL_R_gwfa_SPL_R_gwfa not found. Skipping.
  - Tract IPL_R_gwfa_dPMC_L_gwfa not found. Skipping.
  - Tract IPL_R_gwfa_dPMC_R_gwfa not found. Skipping.
  - Tract IPL_R_gwfa_vPMC_L_gwfa not found. Skipping.
  - Tract IPL_R_gwfa_vPMC_R_gwfa not found. Skipping.
  - Tract IPS_L_gwfa_IPS_R_gwfa not found. Skipping.
  - Tract IPS_L_gwfa_SPL_L_gwfa not fo

  - Tract IPL_L_gwfa_IPL_R_gwfa not found. Skipping.
  - Tract IPL_L_gwfa_IPS_L_gwfa not found. Skipping.
  - Tract IPL_L_gwfa_IPS_R_gwfa not found. Skipping.
  - Tract IPL_L_gwfa_SPL_L_gwfa not found. Skipping.
  - Tract IPL_L_gwfa_SPL_R_gwfa not found. Skipping.
  - Tract IPL_L_gwfa_dPMC_L_gwfa not found. Skipping.
  - Tract IPL_L_gwfa_dPMC_R_gwfa not found. Skipping.
  - Tract IPL_L_gwfa_vPMC_L_gwfa not found. Skipping.
  - Tract IPL_L_gwfa_vPMC_R_gwfa not found. Skipping.
  - Tract IPL_R_gwfa_IPS_L_gwfa not found. Skipping.
  - Tract IPL_R_gwfa_IPS_R_gwfa not found. Skipping.
  - Tract IPL_R_gwfa_SPL_L_gwfa not found. Skipping.
  - Tract IPL_R_gwfa_SPL_R_gwfa not found. Skipping.
  - Tract IPL_R_gwfa_dPMC_L_gwfa not found. Skipping.
  - Tract IPL_R_gwfa_dPMC_R_gwfa not found. Skipping.
  - Tract IPL_R_gwfa_vPMC_L_gwfa not found. Skipping.
  - Tract IPL_R_gwfa_vPMC_R_gwfa not found. Skipping.
  - Tract IPS_L_gwfa_IPS_R_gwfa not found. Skipping.
  - Tract IPS_L_gwfa_SPL_L_gwfa not fo

  - Tract IPL_L_gwfa_IPL_R_gwfa not found. Skipping.
  - Tract IPL_L_gwfa_IPS_L_gwfa not found. Skipping.
  - Tract IPL_L_gwfa_IPS_R_gwfa not found. Skipping.
  - Tract IPL_L_gwfa_SPL_L_gwfa not found. Skipping.
  - Tract IPL_L_gwfa_SPL_R_gwfa not found. Skipping.
  - Tract IPL_L_gwfa_dPMC_L_gwfa not found. Skipping.
  - Tract IPL_L_gwfa_dPMC_R_gwfa not found. Skipping.
  - Tract IPL_L_gwfa_vPMC_L_gwfa not found. Skipping.
  - Tract IPL_L_gwfa_vPMC_R_gwfa not found. Skipping.
  - Tract IPL_R_gwfa_IPS_L_gwfa not found. Skipping.
  - Tract IPL_R_gwfa_IPS_R_gwfa not found. Skipping.
  - Tract IPL_R_gwfa_SPL_L_gwfa not found. Skipping.
  - Tract IPL_R_gwfa_SPL_R_gwfa not found. Skipping.
  - Tract IPL_R_gwfa_dPMC_L_gwfa not found. Skipping.
  - Tract IPL_R_gwfa_dPMC_R_gwfa not found. Skipping.
  - Tract IPL_R_gwfa_vPMC_L_gwfa not found. Skipping.
  - Tract IPL_R_gwfa_vPMC_R_gwfa not found. Skipping.
  - Tract IPS_L_gwfa_IPS_R_gwfa not found. Skipping.
  - Tract IPS_L_gwfa_SPL_L_gwfa not fo

  - Tract IPL_L_gwfa_IPL_R_gwfa not found. Skipping.
  - Tract IPL_L_gwfa_IPS_L_gwfa not found. Skipping.
  - Tract IPL_L_gwfa_IPS_R_gwfa not found. Skipping.
  - Tract IPL_L_gwfa_SPL_L_gwfa not found. Skipping.
  - Tract IPL_L_gwfa_SPL_R_gwfa not found. Skipping.
  - Tract IPL_L_gwfa_dPMC_L_gwfa not found. Skipping.
  - Tract IPL_L_gwfa_dPMC_R_gwfa not found. Skipping.
  - Tract IPL_L_gwfa_vPMC_L_gwfa not found. Skipping.
  - Tract IPL_L_gwfa_vPMC_R_gwfa not found. Skipping.
  - Tract IPL_R_gwfa_IPS_L_gwfa not found. Skipping.
  - Tract IPL_R_gwfa_IPS_R_gwfa not found. Skipping.
  - Tract IPL_R_gwfa_SPL_L_gwfa not found. Skipping.
  - Tract IPL_R_gwfa_SPL_R_gwfa not found. Skipping.
  - Tract IPL_R_gwfa_dPMC_L_gwfa not found. Skipping.
  - Tract IPL_R_gwfa_dPMC_R_gwfa not found. Skipping.
  - Tract IPL_R_gwfa_vPMC_L_gwfa not found. Skipping.
  - Tract IPL_R_gwfa_vPMC_R_gwfa not found. Skipping.
  - Tract IPS_L_gwfa_IPS_R_gwfa not found. Skipping.
  - Tract IPS_L_gwfa_SPL_L_gwfa not fo

  - Tract IPL_L_gwfa_IPL_R_gwfa not found. Skipping.
  - Tract IPL_L_gwfa_IPS_L_gwfa not found. Skipping.
  - Tract IPL_L_gwfa_IPS_R_gwfa not found. Skipping.
  - Tract IPL_L_gwfa_SPL_L_gwfa not found. Skipping.
  - Tract IPL_L_gwfa_SPL_R_gwfa not found. Skipping.
  - Tract IPL_L_gwfa_dPMC_L_gwfa not found. Skipping.
  - Tract IPL_L_gwfa_dPMC_R_gwfa not found. Skipping.
  - Tract IPL_L_gwfa_vPMC_L_gwfa not found. Skipping.
  - Tract IPL_L_gwfa_vPMC_R_gwfa not found. Skipping.
  - Tract IPL_R_gwfa_IPS_L_gwfa not found. Skipping.
  - Tract IPL_R_gwfa_IPS_R_gwfa not found. Skipping.
  - Tract IPL_R_gwfa_SPL_L_gwfa not found. Skipping.
  - Tract IPL_R_gwfa_SPL_R_gwfa not found. Skipping.
  - Tract IPL_R_gwfa_dPMC_L_gwfa not found. Skipping.
  - Tract IPL_R_gwfa_dPMC_R_gwfa not found. Skipping.
  - Tract IPL_R_gwfa_vPMC_L_gwfa not found. Skipping.
  - Tract IPL_R_gwfa_vPMC_R_gwfa not found. Skipping.
  - Tract IPS_L_gwfa_IPS_R_gwfa not found. Skipping.
  - Tract IPS_L_gwfa_SPL_L_gwfa not fo